In [19]:
IMAGE_PATH = '../Data/temp/IMAGE/'  # 이미지 파일들이 들어있는 경로
JSON_PATH = '../Data/temp/JSON/'  # JSON 파일들이 들어있는 경로
SAVE_PATH = '../Data/temp/metadata/'  # 결과 파일을 저장할 경로

In [20]:
import json
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

JSON 파일의 정보를 리스트로 저장

In [21]:
def chart_data(data):
    datatable = []    
    num = len(data)
    
    for n in range(num):
        # 제목, 범례, 데이터 레이블, 단위, 기준 값 추출
        title = data[n]['title'] if data[n]['is_title'] else ''
        legend = data[n]['legend'] if data[n]['is_legend'] else ''
        datalabel = data[n]['data_label'] if data[n]['is_datalabel'] else [0]
        unit = data[n]['unit'] if data[n]['is_unit'] else ''
        base = data[n]['base'] if data[n]['is_base'] else ''
        
        # 원형 차트에는 축 데이터(x_axis, y_axis)가 필요하지 않으므로 제거
        if len(legend) > 1:
            datalabel = np.array(datalabel).transpose().tolist()
        
        # datatable 리스트에 추가
        datatable.append([title, legend, datalabel, unit, base])
        
    return datatable


chart_data()에서 변환된 리스트를 사용할 수 있는 텍스트의 형태로 변환

In [22]:
def datatable(data, chart_type):
    data_table = ''
    num = len(data)
    
    # 원형 차트만 처리
    for n in range(num):
        temp = []
        
        # 제목, 범례, 데이터 레이블, 단위 등을 설정
        title, legend, datalabel, unit, base, x_axis_title, y_axis_title, x_axis, y_axis = data[n]
        legend = [element + f"({unit})" for element in legend]
        
        # 범례가 여러 개 있을 때 처리
        if len(legend) > 1:
            temp.append(f"대상: {base}")
            temp.append(f"제목: {title}")
            temp.append(f"유형: {' '.join(chart_type[0:2])}")
            temp.append(f"{x_axis_title} | {' | '.join(legend)}")
            
            # 원형 차트 데이터 처리
            datalabel = sum(datalabel, [])  # 원형 차트는 데이터가 한 줄로 합쳐짐
            temp.append(f"{' | '.join([str(d) for d in datalabel])}")
            data_table = '\n'.join(temp)
        
        # 범례가 하나일 때 처리
        else:
            temp.append(f"대상: {base}")
            temp.append(f"제목: {title}")
            temp.append(f"유형: {' '.join(chart_type[0:2])}")
            temp.append(f"{x_axis_title} | {unit}")
            
            # 원형 차트의 데이터 레이블 추가
            datalabel = datalabel[0]  # 데이터가 한 개일 때
            for idx, x in enumerate(x_axis):
                row = f"{x} | {str(datalabel[idx])}"
                temp.append(row)
            data_table = '\n'.join(temp)
    
    return data_table

파일을 가져오고, 

In [23]:
def main(IMAGE_PATH, JSON_PATH, SAVE_PATH):
    #이미지 파일을 가져옴
    image_file_list = os.listdir(IMAGE_PATH) 
    
    dataset = []
    for image_file in tqdm(image_file_list):
        #이미지 파일에 대응하는 JSON 파일을 가져옴
        data_id = image_file.split(".jpg")[0]
        json_file_name = image_file.replace("Source", "Label").split(".jpg")[0] + ".json"
        
        with open(JSON_PATH + json_file_name, "r") as f:
            json_data = json.load(f)
        
        #차트 유형 정보를 가져옴
        chart_multi = json_data['metadata']['chart_multi']
        chart_main = json_data['metadata']['chart_main']
        chart_sub = json_data['metadata']['chart_sub']
        chart_type = [chart_multi, chart_sub, chart_main]
        
        #라벨링데이터의 설명문과 요약문을 불러옴
        chart_annotations = json_data['annotations']
        description = "설명: " + json_data["description"].strip()
        summary = "요약: " + "\\".join(json_data["summary"])
        texts = description + "\n" + summary
        
        #차트 데이터를 리스트로 불러오기
        try:
            charData = chart_data(chart_annotations)
        except:
            pass
        
        #차트 데이터를 텍스트로 가져옴
        try:
            dataTable = datatable(charData, chart_type)
            dataset.append([image_file, dataTable, texts, data_id])
        except:
            pass
    
    df = pd.DataFrame(dataset, columns=["file_name", "text", "description", "data_id"])
    
    if not os.path.isdir(SAVE_PATH):
        os.mkdir(SAVE_PATH)
    df.to_csv(SAVE_PATH + "metadata.csv", index=False, header=True)
    print("save")

    return df

In [24]:
main(IMAGE_PATH, JSON_PATH, SAVE_PATH)

100%|██████████| 5/5 [00:00<00:00, 2252.82it/s]

save


,file_name,text,description,data_id
